In [1]:
import sys
sys.path.append("../..")

from time import sleep
from datetime import datetime  # Add this line
import json
import os
from dotenv import load_dotenv
from IPython import display

from sesto.nobitex.client import NobitexClient
from sesto.nobitex.endpoints import NobitexEndpoints

from sesto.utils import convert_lots_to_usd, calculate_position_size, calculate_fee, get_pnl_at_price, get_price_at_pnl
load_dotenv()

True

In [2]:
TRAILING_STOP_STEPS = [
    {'trigger_pnl_multiplier': 4.00, 'new_sl_pnl_multiplier': 3.50},
    {'trigger_pnl_multiplier': 3.50, 'new_sl_pnl_multiplier': 3.00},
    {'trigger_pnl_multiplier': 3.00, 'new_sl_pnl_multiplier': 2.75},
    {'trigger_pnl_multiplier': 2.75, 'new_sl_pnl_multiplier': 2.50},
    {'trigger_pnl_multiplier': 2.50, 'new_sl_pnl_multiplier': 2.25},
    {'trigger_pnl_multiplier': 2.25, 'new_sl_pnl_multiplier': 2.00},
    {'trigger_pnl_multiplier': 2.00, 'new_sl_pnl_multiplier': 1.75},
    {'trigger_pnl_multiplier': 1.75, 'new_sl_pnl_multiplier': 1.50},
    {'trigger_pnl_multiplier': 1.50, 'new_sl_pnl_multiplier': 1.25},
    {'trigger_pnl_multiplier': 1.25, 'new_sl_pnl_multiplier': 1.00},
    {'trigger_pnl_multiplier': 1.00, 'new_sl_pnl_multiplier': 0.75},
    {'trigger_pnl_multiplier': 0.75, 'new_sl_pnl_multiplier': 0.45},
    {'trigger_pnl_multiplier': 0.50, 'new_sl_pnl_multiplier': 0.22},
    {'trigger_pnl_multiplier': 0.25, 'new_sl_pnl_multiplier': 0.12},
    {'trigger_pnl_multiplier': 0.12, 'new_sl_pnl_multiplier': 0.05},
    {'trigger_pnl_multiplier': 0.06, 'new_sl_pnl_multiplier': 0.025},
]

In [3]:
# Jupyter Notebook cell 1: Import necessary modules and set up the client

api_key = os.getenv("NOBITEX_API_KEY_H")
client = NobitexClient(api_key)

In [4]:
positions = client.get_positions()
print(json.dumps(positions['positions'], indent=2))

Requesting URL: https://api.nobitex.ir/positions/list
[
  {
    "id": 8673975,
    "createdAt": "2024-10-02T19:01:00.934488+00:00",
    "side": "sell",
    "srcCurrency": "1m_pepe",
    "dstCurrency": "usdt",
    "status": "Open",
    "marginType": "Isolated Margin",
    "collateral": "33.49959625",
    "leverage": "2",
    "openedAt": "2024-10-02T19:01:01.418958+00:00",
    "closedAt": null,
    "liquidationPrice": "12.6884",
    "entryPrice": "9.325",
    "exitPrice": null,
    "delegatedAmount": "7.1849",
    "liability": "7.1942525283",
    "totalAsset": "100.4116897998",
    "marginRatio": "1.58",
    "liabilityInOrder": "7.1942525283",
    "assetInOrder": "66.24539670583923",
    "unrealizedPNL": "3.4960710741039034",
    "unrealizedPNLPercent": "10.45",
    "expirationDate": "2024-11-02",
    "markPrice": "8.8954",
    "extensionFee": "0"
  }
]


In [5]:
# position = positions['positions'][0]
# oco = client.modify_sl_tp(position['id'], 'oco', position['liability'], 8.71 ,9.58, 9.57)
# print(json.dumps(oco, indent=2))

In [6]:
orders = client.get_my_orders()
print(json.dumps(orders['orders'], indent=2))

Requesting URL: https://api.nobitex.ir/market/orders/list
[
  {
    "type": "buy",
    "execution": "StopLimit",
    "tradeType": "Margin",
    "srcCurrency": "1M_Pepe",
    "dstCurrency": "Tether",
    "price": "9.2081",
    "amount": "7.1942525283",
    "totalPrice": "66.24539670583923",
    "totalOrderPrice": "66.24539670583923",
    "matchedAmount": "0",
    "unmatchedAmount": "7.1942525283",
    "clientOrderId": null,
    "isMyOrder": false,
    "param1": "9.2081391",
    "pairId": 1964997268,
    "leverage": "2",
    "side": "close"
  },
  {
    "type": "buy",
    "execution": "Limit",
    "tradeType": "Margin",
    "srcCurrency": "1M_Pepe",
    "dstCurrency": "Tether",
    "price": "2",
    "amount": "7.1942525283",
    "totalPrice": "14.3885050566",
    "totalOrderPrice": "14.3885050566",
    "matchedAmount": "0",
    "unmatchedAmount": "7.1942525283",
    "clientOrderId": null,
    "isMyOrder": false,
    "pairId": 1964997270,
    "leverage": "2",
    "side": "close"
  }
]


In [7]:
# all_orders = client.cancel_all_orders()
# print(json.dumps(all_orders, indent=2))

In [8]:
def calculate_trade_capital(symbol, volume_lots, leverage, price_open):
    position_size_usd = convert_lots_to_usd(symbol, volume_lots, price_open)
    capital_used = position_size_usd / leverage
    return capital_used

def has_stop_loss_order(src_currency, dst_currency):
    for order in orders['orders']:
        isSrcCurrencyMatch = order['srcCurrency'].lower() == src_currency.lower()
        isDstCurrencyMatch = order['dstCurrency'].lower() == dst_currency.lower() or (dst_currency.lower() == 'usdt' and order['dstCurrency'].lower() == 'tether')
        if order['execution'] == 'StopLimit' and isSrcCurrencyMatch and isDstCurrencyMatch:
            return True
    return False

def get_stop_loss_order(src_currency, dst_currency):
    for order in orders['orders']:
        isSrcCurrencyMatch = order['srcCurrency'].lower() == src_currency.lower()
        isDstCurrencyMatch = order['dstCurrency'].lower() == dst_currency.lower() or (dst_currency.lower() == 'usdt' and order['dstCurrency'].lower() == 'tether')
        if order['execution'] == 'StopLimit' and isSrcCurrencyMatch and isDstCurrencyMatch:
            return order
    return None



In [9]:
while True:
    for position in positions['positions']:
        symbol = position['srcCurrency'] + position['dstCurrency']
        created_at = datetime.fromisoformat(position['createdAt'])
        leverage=float(position['leverage'])
        entryPrice=float(position['entryPrice'])
        sl_order = get_stop_loss_order(position['srcCurrency'], position['dstCurrency'])
        collateral = float(position['collateral'])
        unrealized_pnl = float(position['unrealizedPNL'])
        trade_size = calculate_position_size(collateral, leverage)
        fee = calculate_fee(position_size_usd=trade_size) 
        
        current_pnl_percentage = (unrealized_pnl / collateral) * 100
        print(f"{created_at} - {symbol} - PNL INFO - PNL: ${unrealized_pnl:.3f} or {current_pnl_percentage:.3f}%")

        if sl_order:
            sl_price = float(sl_order['price'])
            current_sl_pnl = get_pnl_at_price(sl_price, entryPrice, trade_size, leverage, 'long' if position['side'] == 'buy' else 'short')
            current_sl_pnl_percentage = (current_sl_pnl / collateral) * 100
            print(f"{created_at} - {symbol} - POSITION INFO - OPEN PRICE: ${entryPrice:.3f} - TRADE CAPITAL: ${collateral:.3f} - CURRENT SL: ${sl_price:.3f}")        
            print(f"{created_at} - {symbol} - SL PNL INFO - PNL AT CURRENT SL: ${current_sl_pnl:.3f} or {current_sl_pnl_percentage:.3f}%")
        
        
        for trailing_step in TRAILING_STOP_STEPS:
            trigger_pnl_multiplier = trailing_step['trigger_pnl_multiplier']
            new_sl_pnl_multiplier = trailing_step['new_sl_pnl_multiplier']

            price_at_trigger_pnl_multiplier = get_price_at_pnl(
                pnl_multiplier=trigger_pnl_multiplier,
                order_fee=fee,
                position_size_usd=trade_size,
                leverage=leverage,
                entry_price=entryPrice,
                type='long' if position['side'] == 'buy' else 'short'
            )
            new_sl_at_new_sl_pnl_multiplier = get_price_at_pnl(
                pnl_multiplier=new_sl_pnl_multiplier,
                order_fee=fee,
                position_size_usd=trade_size,
                leverage=leverage,
                entry_price=entryPrice,
                type='long' if position['side'] == 'buy' else 'short'
            )
            pnl_at_trigger = get_pnl_at_price(
                price_at_trigger_pnl_multiplier,
                entryPrice,
                trade_size,
                leverage,
                'long' if position['side'] == 'buy' else 'short'
            )
            pnl_at_new_sl = get_pnl_at_price(
                new_sl_at_new_sl_pnl_multiplier,
                entryPrice,
                trade_size,
                leverage,
                'long' if position['side'] == 'buy' else 'short'
            )

            pnl_threshold = trigger_pnl_multiplier * collateral

            if unrealized_pnl >= pnl_threshold:
                old_sl_price = float(sl_order['price']) if sl_order else None

                if position['side'] == 'buy':  # Long
                    new_sl_price = get_price_at_pnl(
                        pnl_multiplier=new_sl_pnl_multiplier,
                        order_fee=fee,
                        position_size_usd=collateral,
                        leverage=leverage,
                        entry_price=entryPrice,
                        type='long'
                    )
                    new_tp_price = get_price_at_pnl(
                        pnl_multiplier=5,
                        order_fee=fee,
                        position_size_usd=collateral,
                        leverage=leverage,
                        entry_price=entryPrice,
                        type='long'
                    )
                    if old_sl_price is None or new_sl_price > old_sl_price:
                        print(f'{created_at} - TRIGGERED TRAILING STOP - {position.symbol} - NEW SL: ${new_sl_price:.3f}')
                        client.cancel_all_orders(trade_type='margin', src_currency=position['srcCurrency'], dst_currency=position['dstCurrency'])
                        oco = client.modify_sl_tp(position['id'], 'oco', position['liability'], new_tp_price,new_sl_price, new_sl_price)
                        print(oco)
                else:  # Short
                    new_sl_price = get_price_at_pnl(
                        pnl_multiplier=new_sl_pnl_multiplier,
                        order_fee=fee,
                        position_size_usd=collateral,
                        leverage=leverage,
                        entry_price=entryPrice,
                        type='short'
                    )
                    new_tp_price = get_price_at_pnl(
                        pnl_multiplier=5,
                        order_fee=fee,
                        position_size_usd=collateral,
                        leverage=leverage,
                        entry_price=entryPrice,
                        type='short'
                    )
                    if old_sl_price is None or new_sl_price < old_sl_price:
                        print(f'{created_at} - TRIGGERED TRAILING STOP - {symbol} - NEW SL: ${new_sl_price:.3f}')
                        client.cancel_all_orders(trade_type='margin', src_currency=position['srcCurrency'], dst_currency=position['dstCurrency'])
                        oco = client.modify_sl_tp(position['id'], 'oco', position['liability'], new_tp_price,new_sl_price, new_sl_price)
                        print(oco)
                
                break
        sleep(10)

2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - PNL INFO - PNL: $3.496 or 10.436%
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - POSITION INFO - OPEN PRICE: $9.325 - TRADE CAPITAL: $33.500 - CURRENT SL: $9.208
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - SL PNL INFO - PNL AT CURRENT SL: $0.840 or 2.507%
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - PNL INFO - PNL: $3.496 or 10.436%
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - POSITION INFO - OPEN PRICE: $9.325 - TRADE CAPITAL: $33.500 - CURRENT SL: $9.208
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - SL PNL INFO - PNL AT CURRENT SL: $0.840 or 2.507%
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - PNL INFO - PNL: $3.496 or 10.436%
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - POSITION INFO - OPEN PRICE: $9.325 - TRADE CAPITAL: $33.500 - CURRENT SL: $9.208
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - SL PNL INFO - PNL AT CURRENT SL: $0.840 or 2.507%
2024-10-02 19:01:00.934488+00:00 - 1m_pepeusdt - PNL INFO - PNL: